In [2]:
import threading
import requests
import time
from flask import Flask, request, jsonify
from werkzeug.serving import make_server

# Step 1: Setting up the local Flask server
class ServerThread(threading.Thread):
    def __init__(self, app):
        threading.Thread.__init__(self)
        self.server = make_server('localhost', 8080, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print("Starting Flask server on port 8080")
        self.server.serve_forever()

    def shutdown(self):
        self.server.shutdown()
        print("Flask server shutdown")

app = Flask(__name__)

@app.route('/', methods=['GET'])
def hello_world():
    return 'Hello, world!', 200

@app.route('/', methods=['POST'])
def echo_post():
    data = request.form.to_dict()
    return jsonify(data), 200

# Start the server in a separate thread
server = ServerThread(app)
server.start()

# Allow some time for the server to start up
time.sleep(1)

# Initialize a flag to track the success of all steps
all_steps_successful = True

# Step 2: Importing the necessary package and checking if 'requests' is installed
try:
    import requests
    print("Step 2: Importing 'requests' package passed.")
except ImportError as e:
    all_steps_successful = False
    print(f"Step 2 failed: {e}")

# Step 3: Sending a simple GET request to verify basic functionality
try:
    response = requests.get("http://localhost:8080")
    assert response.status_code == 200, "Expected status code 200"
    print("Step 3: Simple GET request passed.")
except Exception as e:
    all_steps_successful = False
    print(f"Step 3 failed: {e}")

# Step 4: Testing error handling with a nonexistent domain
try:
    response = requests.get("http://localhost:9999")  # Unused port
    response.raise_for_status()
    print("Step 4 failed: Expected an HTTPError but didn't get one.")
except requests.exceptions.RequestException as e:
    print(f"Step 4: Correctly handled connection error for nonexistent domain: {e}")
except Exception as e:
    all_steps_successful = False
    print(f"Step 4 failed: {e}")

# Step 5: Sending a POST request with data
try:
    data = {"key": "value"}
    response = requests.post("http://localhost:8080", data=data)
    assert response.status_code == 200, "Expected status code 200"
    print("Step 5: POST request with data passed.")
except Exception as e:
    all_steps_successful = False
    print(f"Step 5 failed: {e}")

# Step 6: Testing timeout handling
try:
    response = requests.get("http://localhost:8080", timeout=0.001)
    print("Step 6 failed: Expected a timeout exception but didn't get one.")
except requests.exceptions.Timeout as e:
    print(f"Step 6: Timeout exception correctly raised: {e}")
except Exception as e:
    all_steps_successful = False
    print(f"Step 6 failed: {e}")

# Step 7: Handling redirects (local server doesn't handle redirects; skipping)

# Step 8: Testing JSON response parsing
try:
    response = requests.post("http://localhost:8080", data={"key": "value"})
    json_data = response.json()
    assert "key" in json_data, "Expected key 'key' in JSON response"
    print("Step 8: JSON response parsing passed.")
except Exception as e:
    all_steps_successful = False
    print(f"Step 8 failed: {e}")

# Final Confirmation
if all_steps_successful:
    print("All extensive tests for the 'requests' package completed successfully.")
else:
    print("One or more steps failed. Please review the errors above.")

# Step 9: Shutting down the server
server.shutdown()


Starting Flask server on port 8080


127.0.0.1 - - [31/Aug/2024 21:48:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 21:48:38] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 21:48:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Aug/2024 21:48:38] "POST / HTTP/1.1" 200 -


Step 2: Importing 'requests' package passed.
Step 3: Simple GET request passed.
Step 4: Correctly handled connection error for nonexistent domain: HTTPConnectionPool(host='localhost', port=9999): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xffff8d91a490>: Failed to establish a new connection: [Errno 111] Connection refused'))
Step 5: POST request with data passed.
Step 6 failed: Expected a timeout exception but didn't get one.
Step 8: JSON response parsing passed.
All extensive tests for the 'requests' package completed successfully.
Flask server shutdown
